<a href="https://colab.research.google.com/github/seosumin/start-github/blob/master/cnn_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import TensorFlow

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# 이미지를 어떻게 입력받는지에 따른 코드 분할

# 1) 바로 array 형태의 이미지 데이터를 받을 때
# 2) 이미지 파일이 path에 담겨있고 이미지 형태일 때

In [15]:
# cifar10 데이터 다운로드 해당 데이터는 다운받는 순간부터 3차원 형태를 가지고 있다.
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# 255로 스케일링 하자 
#train_images, test_images = train_images / 255.0, test_images / 255.0

In [34]:
# 만약 시험이 위의 데이터처럼 어레이 형식이 아니라 이미지 형태라면..?
# 데이터 path가 있는 경우라면, imageGenerator를 써보자.
# 그런데 이 코드가 먹힐라면, path에 클래스별로 따로 하위 폴더를 가지고 있어야 한다. 
# 참고 링크 https://tykimos.tistory.com/13


from tensorflow.keras.preprocessing.image import ImageDataGenerator
# imageDataGenerator 호출 
train_datagen = ImageDataGenerator(rescale=1/255,
                                  validation_split=0.2)

#.flow_from_directory(만약 path를 안다면?), target 싸이즈를 지정해준다. 
train_generator = train_datagen.flow_from_directory(
  os.path.join(os.getcwd(),photo_dir), # 경로 설정
  target_size=(299, 299),
  batch_size=32,
  class_mode='binary', #다중문제는 categorical
  subset='training' # 해당 파라미터를 통해 train/valid 분할 가능
)

#valid
valid_generator = train_datagen.flow_from_directory(
  os.path.join(os.getcwd(),photo_dir), #경로 설정
  target_size=(299, 299),
  batch_size=32,
  class_mode='binary', #다중문제는 categorical
  subset='validation'
)

SyntaxError: ignored

In [35]:
# 어레이로 받았을 경우에도 imageDataGenerater를 쓸수있다.
# .flow를 활용할 수 있다. 
# 참고 링크 https://keras.io/ko/preprocessing/image/
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split=0.2)

train_generator = train_datagen.flow(
    train_images, train_labels,
    batch_size=32,
    subset='training' # 해당 파라미터를 통해 train/valid 분할 가능
)

valid_generator = train_datagen.flow(
   train_images, train_labels,
  batch_size=32,
  subset='validation'
)

print(train_generator.x.shape)
print(valid_generator.x.shape)

(40000, 32, 32, 3)
(10000, 32, 32, 3)


In [39]:
train_generator.y

array([[1],
       [6],
       [6],
       ...,
       [9],
       [1],
       [1]], dtype=uint8)

In [32]:
#test
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow(
    test_images, test_labels,
    batch_size=32)

In [52]:
clear_session()
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0

In [53]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels), verbose=1)

Epoch 1/10
1563/1563 [==============================] - 10s 5ms/step - loss: 1.7118 - accuracy: 0.3917 - val_loss: 1.4115 - val_accuracy: 0.4895
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.3110 - accuracy: 0.5336 - val_loss: 1.2477 - val_accuracy: 0.5556
Epoch 3/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1645 - accuracy: 0.5920 - val_loss: 1.2680 - val_accuracy: 0.5687
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0604 - accuracy: 0.6302 - val_loss: 1.0964 - val_accuracy: 0.6216
Epoch 5/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9834 - accuracy: 0.6579 - val_loss: 1.1016 - val_accuracy: 0.6164
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9190 - accuracy: 0.6778 - val_loss: 1.0683 - val_accuracy: 0.6476
Epoch 7/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8533 - accuracy: 0.7044 - val_loss: 1.0867 - val_accuracy

In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Dropout, Input, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from keras import backend as K


In [43]:
base_model_1 = tf.keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=[32,32,3],
    pooling=None,
    #classes=10,
    #classifier_activation="softmax",
)

In [44]:
base_model_1.trainable= False #vgg16 동결하기

In [45]:
base_model_1.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [46]:
inputs = tf.keras.Input(shape=(32, 32, 3))
x = base_model_1(inputs, training=False)
x = tf.keras.layers.Flatten()(x)
dense1= tf.keras.layers.Dense(300, activation='relu')(x)
outputs = tf.keras.layers.Dense(10,activation='softmax')(dense1)
model = tf.keras.Model(inputs, outputs)

In [47]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [48]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 300)               153900    
                                                                 
 dense_1 (Dense)             (None, 10)                3010      
                                                                 
Total params: 14,871,598
Trainable params: 156,910
Non-trainable params: 14,714,688
_________________________________________________________________


In [49]:
es= EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1, restore_best_weights=True)
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    verbose=1,
    callbacks=[es])

Epoch 1/30
1250/1250 [==============================] - 25s 12ms/step - loss: 1.3770 - accuracy: 0.1003 - val_loss: 1.2345 - val_accuracy: 0.1055
Epoch 2/30
1250/1250 [==============================] - 15s 12ms/step - loss: 1.1873 - accuracy: 0.0993 - val_loss: 1.1844 - val_accuracy: 0.1215
Epoch 3/30
1250/1250 [==============================] - 15s 12ms/step - loss: 1.1188 - accuracy: 0.0985 - val_loss: 1.1452 - val_accuracy: 0.1083
Epoch 4/30
1250/1250 [==============================] - 14s 12ms/step - loss: 1.0681 - accuracy: 0.0986 - val_loss: 1.1175 - val_accuracy: 0.0868
Epoch 5/30
1250/1250 [==============================] - 14s 12ms/step - loss: 1.0243 - accuracy: 0.0988 - val_loss: 1.1315 - val_accuracy: 0.1117
Epoch 6/30
1250/1250 [==============================] - 15s 12ms/step - loss: 0.9841 - accuracy: 0.0989 - val_loss: 1.1145 - val_accuracy: 0.0952
Epoch 7/30
1250/1250 [==============================] - 14s 12ms/step - loss: 0.9462 - accuracy: 0.0979 - val_loss: 1.0877 -